In [1]:
import cv2
import mediapipe as mp

import pickle
import numpy as np

model_dic = pickle.load(open('.\model.p', 'rb'))
model = model_dic['model']

cap = cv2.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp. solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

while True:
    data_aux=[]
    ret, frame = cap.read()
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
            
        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x)
                data_aux.append(y)
         
        model.predict([np.asarray(data_aux)])
        
    cv2.imshow('frame', frame)
    cv2.waitKey(25)
    
cap.release()
cv2.destroyAllWindows()

In [ ]:
!pip install mediapipe==0.10.1

In [2]:
import pickle
import numpy as np

# Load the model
model_dic = pickle.load(open('.\\model.p', 'rb'))
model = model_dic['model']

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Mediapipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

# Number of features the model expects (2 hands × 42 features each = 84 features)
expected_features = 84

while True:
    data_aux = []
    ret, frame = cap.read()

    if not ret:
        print("Error: Unable to capture frame.")
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe
    results = hands.process(frame_rgb)

    prediction_text = "No hands detected"  # Default text when no hands are detected

    if results.multi_hand_landmarks:
        # Sort hand landmarks by visibility (z-index) to maintain consistency
        hands_list = sorted(
            results.multi_hand_landmarks, key=lambda hand: hand.landmark[0].z
        )

        # Process up to 2 hands
        for hand_landmarks in hands_list[:2]:  # Take at most two hands
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x)
                data_aux.append(y)

        # If fewer than 2 hands detected, pad features for the missing hand
        if len(data_aux) < expected_features:
            data_aux.extend([0] * (expected_features - len(data_aux)))

        # Convert to NumPy array and reshape
        input_features = np.asarray(data_aux).reshape(1, -1)

        # Predict using the model
        prediction = model.predict(input_features)
        prediction_text = f"Prediction: {prediction[0]}"  # Update prediction text

        # Draw landmarks
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style(),
            )

    # Overlay the prediction text on the frame
    cv2.putText(
        frame,
        prediction_text,
        (10, 50),  # Position (x, y)
        cv2.FONT_HERSHEY_SIMPLEX,
        1,  # Font scale
        (0, 255, 0),  # Text color (green)
        2,  # Thickness
        cv2.LINE_AA,
    )

    # Display the frame
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [ ]:
print("Labels:", model.classes_)
